In [1]:
import glob
import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 데이터 다운로드

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


---

# 데이터 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z0-9?.!,¿]+", " ", sentence)
    sentence = re.sub(r"([()])", r"", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue

    preprocessed_sentence = preprocess_sentence(sentence)

    # 단어 개수가 15개를 넘어가면 학습 데이터에서 제외
    if len(preprocessed_sentence.split()) > 15:
        continue
        
    corpus.append(preprocessed_sentence)

In [5]:
len(corpus)

156064

In [6]:
def tokenize(corpus, num_words=12000):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=num_words, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2983 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  259  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fbeec3a6460>


In [7]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

In [8]:
enc_train, enc_validation, dec_train, dec_validation = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True,
                                                          random_state=7)

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124851, 14)
Target Train: (124851, 14)


# 모델 만들기

### 데이터셋 (train & validation)

In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)

validation_dataset = tf.data.Dataset.from_tensor_slices((enc_validation, dec_validation))
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=False)

### model 작성

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

### model 컴파일

In [15]:
embedding_size = 512 # 워드 벡터의 차원수 (추상적인 특징을 표현하는 크기) 
hidden_size = 2048 # 일꾼
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

### model 학습시키기

In [16]:
model.fit(train_dataset,
          epochs=10,
          validation_data=validation_dataset)

Epoch 1/10
488/488 [==============================] - 273s 521ms/step - loss: 3.3180 - val_loss: 2.9314
Epoch 2/10
488/488 [==============================] - 255s 522ms/step - loss: 2.7860 - val_loss: 2.6976
Epoch 3/10
488/488 [==============================] - 255s 522ms/step - loss: 2.5031 - val_loss: 2.5279
Epoch 4/10
488/488 [==============================] - 254s 521ms/step - loss: 2.2235 - val_loss: 2.3917
Epoch 5/10
488/488 [==============================] - 255s 522ms/step - loss: 1.9507 - val_loss: 2.2888
Epoch 6/10
488/488 [==============================] - 255s 523ms/step - loss: 1.6988 - val_loss: 2.2183
Epoch 7/10
488/488 [==============================] - 255s 522ms/step - loss: 1.4810 - val_loss: 2.1725
Epoch 8/10
488/488 [==============================] - 254s 521ms/step - loss: 1.3012 - val_loss: 2.1491
Epoch 9/10
488/488 [==============================] - 255s 521ms/step - loss: 1.1644 - val_loss: 2.1541
Epoch 10/10
488/488 [==============================] - 255s 522m

### model 평가

In [19]:
validation_steps = 20
result = model.evaluate(validation_dataset, steps=validation_steps)

print(result)

20/20 [==============================] - 3s 169ms/step - loss: 2.1941
2.1941232681274414


### model 테스트 - 예측돌리기

In [21]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text(model, tokenizer, init_sentence="<start> i need", max_len=20)

'<start> i need you to hurry up now <end> '

# 회고

### 1. 어려웠던 점
- 모델을 학습시키는데 적절한 하이퍼 파라미터를 찾는 것이 어려웠습니다.
- 데이터의 shape이 이해가 잘 안돼서 어떻게 배치, 토큰, 확률값 등이 연결되는지 어려웠습니다. 


### 3. 시도한 것들
- BatchDataset과 TakeDataset, 그리고 학습에 적용된 데이터셋, shape와 배치사이즈를 print하면서 이해했습니다. 
    - 원래 전체 데이터 shape : (24015, 20)
    - 배치 사이즈 : 256
    - 학습 도는 횟수 : 93
    - model(dataset.take(1)) shape : (256, 20, 7001)
        - 256 : 배치 사이즈
        - 20 : 문장 길이(단어 갯수)
        - 7001 : 특정 단어에 대해 단어장의 7001개 단어들의 연관성 확률이 가장 높은지 모델링해서 나온 차원
- 데이터셋이 배치 사이즈만큼 나뉘어 학습을 돌 때, 남는데이터셋에 대해서 제거되어 실제 데이터가 원본보다 적어지는 것을 발견했습니다. 데이터가 누락되지 않도록 dataset.batch 함수의 drop_remainder=False로 하고 돌렸더니 정상적으로 모든 데이터에 대해 처리되는 것을 확인할 수 있었습니다.
- validation loss 값을 낮추기 위해, embedding_size(256 -> 512)와 hidden_size(1024 -> 2048)를 늘려서 돌렸습니다.


### 5. 자기 다짐
- model 학습의 val_loss를 줄이는것에 집중하다가, 그간 뭘했는지 기억이 도통 나질 않았습니다. 회고는 그렇다 쳐도... 돌이켜보면 어제 배운걸 또 까먹고 지난주에 배운걸 또 까먹고 있습니다. 매일 수업 전, 전날 배운걸 빠르게 복습해야겠다고 다짐하게 됐습니다. 회고도 가능하면 그날 바로 작성해야겠습니다.